In [1]:
# importing all the required libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

import IPython.display as ipd

In [2]:
np.random.seed(6)

In [3]:
def play(audio, rate=16000):
  ipd.display(ipd.Audio(audio, rate=rate))

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# constants
SAMPLE_RATE = 16000

In [19]:
import glob

In [20]:
FILEPATH = 'drive/MyDrive/DATA/RAVDESS-dataset/Audio*/*/*.wav'
all_audio_files = sorted(glob.glob(FILEPATH))
len(all_audio_files)

2496

In [21]:
all_audio_files[:20]

['drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-01-01-01-01-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-01-01-01-02-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-01-01-02-01-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-01-01-02-02-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-01-01-01-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-01-01-02-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-01-02-01-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-01-02-02-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-02-01-01-01.wav',
 'drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs_Actors_01-24/Actor_01/03-02-02-02-01-02-01.wav',
 'drive/MyDrive/DATA

In [22]:
from tqdm import tqdm
import os

In [23]:
label_to_emotion = {'01':'neutral', 
                    '02': 'calm', 
                    '03': 'happy',
                    '04': 'sad',
                    '05': 'angry',
                    '06': 'fearful',
                    '07': 'disgust',
                    '08': 'surprised'}

In [24]:
FILES = {'filepath':[], 
         'label':[], 
         'emotion': [],
         'actor':[], 
         'intensity': [], 
         'duration': []}
want_duration = False
for file in tqdm(all_audio_files):
  filename = file.replace('.wav', '')
  label = os.path.basename(filename).split('-')[2]
  intensity = os.path.basename(filename).split('-')[3]
  actor = os.path.basename(filename).split('-')[-1]
  
  # filling the dictionary
  FILES['filepath'].append(file)
  FILES['label'].append(label)
  FILES['emotion'].append(label_to_emotion[label])
  FILES['actor'].append(actor)
  FILES['intensity'].append(intensity)
  if want_duration:
    FILES['duration'].append(librosa.get_duration(filename=file))
  else:
    FILES['duration'].append(0.0)

100%|██████████| 2496/2496 [00:00<00:00, 220121.59it/s]


In [25]:
df = pd.DataFrame(FILES)
df

,filepath,label,emotion,actor,intensity,duration
0,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs...,01,neutral,01,01,0.0
1,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs...,01,neutral,01,01,0.0
2,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs...,01,neutral,01,01,0.0
3,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs...,01,neutral,01,01,0.0
4,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Songs...,02,calm,01,01,0.0
...,...,...,...,...,...,...
2491,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Speec...,08,surprised,24,01,0.0
2492,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Speec...,08,surprised,24,02,0.0
2493,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Speec...,08,surprised,24,02,0.0
2494,drive/MyDrive/DATA/RAVDESS-dataset/Audio_Speec...,08,surprised,24,02,0.0


In [26]:
from collections import Counter

Counter(df.emotion)

Counter({'angry': 384,
         'calm': 384,
         'disgust': 192,
         'fearful': 384,
         'happy': 384,
         'neutral': 192,
         'sad': 384,
         'surprised': 192})

In [27]:
# to split the data into training and validation
from sklearn.model_selection import train_test_split

In [28]:
train_df, dev_df = train_test_split(df, shuffle=True, stratify=df['label'], test_size=0.15, random_state=6)
dev_df, test_df = train_test_split(dev_df, shuffle=True, stratify=dev_df['label'], test_size=0.5, random_state=6)

In [29]:
train_df.shape, dev_df.shape, test_df.shape

((2121, 6), (187, 6), (188, 6))

In [30]:
print(Counter(train_df.emotion))
print(Counter(dev_df.emotion))
print(Counter(test_df.emotion))

Counter({'fearful': 327, 'angry': 327, 'sad': 326, 'happy': 326, 'calm': 326, 'surprised': 163, 'disgust': 163, 'neutral': 163})
Counter({'happy': 29, 'sad': 29, 'calm': 29, 'angry': 28, 'fearful': 28, 'disgust': 15, 'surprised': 15, 'neutral': 14})
Counter({'sad': 29, 'angry': 29, 'fearful': 29, 'calm': 29, 'happy': 29, 'neutral': 15, 'surprised': 14, 'disgust': 14})


In [32]:
class AudioDatasetMean(torch.utils.data.Dataset):
  def __init__(self, df):
    self.df = df
    self.cache = {}

  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self, idx):
    if idx in self.cache:
      return self.cache[idx]
    file = self.df.iloc[idx]
    label = int(file.label)-1
    audio_duration = librosa.get_duration(filename=file.filepath)
    audio, sr = librosa.load(file.filepath, 
                             sr=SAMPLE_RATE)
    # melspectrogram
    mel_spec = librosa.feature.melspectrogram(audio, sr=SAMPLE_RATE).mean(1)
    # mel_spec = librosa.feature.melspectrogram(audio)
    self.cache[idx] = [torch.Tensor(mel_spec), torch.Tensor([label])]
    return self.cache[idx]
    # return torch.Tensor(mel_spec), torch.Tensor([label])


class AudioDatasetMeanInitLoaded(torch.utils.data.Dataset):
  def __init__(self, df):
    self.df = df
    self.data = []
    for i, line in tqdm(self.df.iterrows(), total=self.df.shape[0]):
      label = int(line.label)-1
      audio, sr = librosa.load(line.filepath, sr=SAMPLE_RATE)
      # melspectrogram
      mel_spec_mean = librosa.feature.melspectrogram(audio, sr=SAMPLE_RATE).mean(1)
      self.data.append([audio, mel_spec_mean, label])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    audio, mel_spec_mean, label = self.data[idx]
    return torch.Tensor(mel_spec_mean), torch.Tensor([label])

In [33]:
train_dataset = AudioDatasetMeanInitLoaded(df=train_df)
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                               shuffle=True, 
                                               batch_size=128, 
                                               num_workers=8)


dev_dataset = AudioDatasetMeanInitLoaded(df=dev_df)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, 
                                             shuffle=False, 
                                             batch_size=32, 
                                             num_workers=8)

test_dataset = AudioDatasetMeanInitLoaded(df=test_df)
test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                             shuffle=False, 
                                             batch_size=32, 
                                             num_workers=8)


100%|██████████| 188/188 [01:42<00:00,  1.84it/s]


In [34]:
print(len(train_dataset), len(dev_dataset), len(test_dataset))
print(len(train_dataloader), len(dev_dataloader), len(test_dataloader))

2121 187 188
17 6 6


In [70]:
class MLPClass(torch.nn.Module):
  def __init__(self, input_features=13, n_hidden=512, num_classes=8):
    super().__init__()
    p = 0.0
    self.model = nn.Sequential(nn.Linear(input_features, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p), nn.ReLU(),
                                   nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p), nn.ReLU(),
                                  nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p), nn.ReLU(),
                                  nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p), nn.ReLU(),
                                  nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p), nn.ReLU(),
                                   nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p),  nn.ReLU(),
                                   nn.Linear(n_hidden, n_hidden), nn.LayerNorm(n_hidden), nn.Dropout(p=p),  nn.ReLU(),
                                   nn.Linear(n_hidden, num_classes))
  def forward(self, X):

    return self.model(X)

In [71]:
INPUT_FEATURES = 128
N_HIDDEN = 512
NUM_CLASSES = 8

model_dnn_v1 = MLPClass(INPUT_FEATURES, 
                        N_HIDDEN, 
                        NUM_CLASSES)

In [72]:
model_dnn_v1

MLPClass(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): Dropout(p=0.0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (6): Dropout(p=0.0, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (10): Dropout(p=0.0, inplace=False)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (14): Dropout(p=0.0, inplace=False)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (18): Dropout(p=0.0, inplace=False)
    (19): ReLU()
    (20): Linear(in_features=512, out_features=512, bias=

In [73]:
DEVICE = 'cuda'
model_dnn_v1.to(DEVICE)

MLPClass(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): Dropout(p=0.0, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (6): Dropout(p=0.0, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (10): Dropout(p=0.0, inplace=False)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (14): Dropout(p=0.0, inplace=False)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (18): Dropout(p=0.0, inplace=False)
    (19): ReLU()
    (20): Linear(in_features=512, out_features=512, bias=

In [74]:
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model_dnn_v1.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.5, patience=1)

In [75]:
def get_accuracy(yhat, y):
  return (yhat.argmax(1)==y).float().mean()


def train_one_epoch_dnn(model, dataset, criterion, optim, epoch):
  model.train()
  losses = []
  accus = []
  stime = time.time()
  pbar = tqdm(dataset)
  for i, batch in enumerate(pbar):
    feats = batch[0].to(DEVICE)
    labels = batch[1].flatten().to(DEVICE)

    outs = model(feats)

    optim.zero_grad()
    l = criterion(outs, labels.long())
    l.backward()
    optim.step()

    losses.append(l.item())
    accu = get_accuracy(outs, labels)
    accus.append(accu.item())

    accuracy_avg = sum(accus)/len(accus)
    loss_avg = sum(losses)/len(losses)
    pbar.set_description(f"Progress - Accu: {accuracy_avg:.3f}\tLoss: {loss_avg:.3f}")

  etime = time.time()
  accuracy_avg = sum(accus)/len(accus)
  loss_avg = sum(losses)/len(losses)
  time_total = etime - stime
  print(f'[TRAIN] Epoch: {epoch}\tAccu: {accuracy_avg}\tLoss: {loss_avg}\tTime: {time_total:.3f} secs')
  return loss_avg, accuracy_avg, time_total

@torch.no_grad()
def evaluate_model(model, dataset, criterion, epoch):
  model.eval()
  losses = []
  accus = []
  stime = time.time()
  for i, batch in enumerate(dataset):
    feats = batch[0].to(DEVICE)
    labels = batch[1].flatten().to(DEVICE)

    outs = model(feats)

    l = criterion(outs, labels.long())

    losses.append(l.item())
    accu = get_accuracy(outs, labels)
    accus.append(accu.item())
  etime = time.time()
  accuracy_avg = sum(accus)/len(accus)
  loss_avg = sum(losses)/len(losses)
  time_total = etime - stime
  print(f'[VALID] Epoch: {epoch}\tAccu: {accuracy_avg}\tLoss: {loss_avg}\tTime: {time_total:.3f} secs')
  return loss_avg, accuracy_avg, time_total

In [76]:
# for i, batch in enumerate(dev_dataloader):
#   print(batch)
#   break

In [77]:
# batch[0].shape

In [78]:
get_lr = lambda optim: optim.param_groups[0]['lr']

In [79]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
N_EPOCHS = 50
best_training_loss = 1000
best_validation_loss = 1000

best_training_accuracy = 0
best_validation_accuracy = 0

best_training_epoch = -1
best_validation_epoch = -1
STIME = time.time()
for epoch in range(1, N_EPOCHS + 1):
  # training the model for one epoch
  print(f"Starting Training epoch: {epoch}")
  train_loss_avg, train_accuracy_avg, train_time_total = train_one_epoch_dnn(model_dnn_v1, train_dataloader, criterion, optim, epoch)

  # evaluate the model one epoch
  print(f"Starting Evaluation epoch: {epoch}")
  dev_loss_avg, dev_accuracy_avg, dev_time_total = evaluate_model(model_dnn_v1, dev_dataloader, criterion, epoch)

  # scheduler
  scheduler.step(dev_loss_avg)
  print(f"Learning rate at epoch: {epoch} is {get_lr(optim)}")

  if train_loss_avg < best_training_loss:
    best_training_loss = train_loss_avg
    print(f"Best training loss updated at epoch: {epoch} as {best_training_loss}")
  
  if dev_loss_avg < best_validation_loss:
    best_validation_loss = dev_loss_avg
    print(f"Best validation loss updated at epoch: {epoch} as {dev_loss_avg}")
  
  if train_accuracy_avg > best_training_accuracy:
    best_training_accuracy = train_accuracy_avg
    best_training_epoch = epoch
    print(f"Best training accuracy updated at epoch: {epoch} as {best_training_loss}")
  
  if dev_accuracy_avg > best_validation_accuracy:
    best_validation_accuracy = dev_accuracy_avg
    best_validation_epoch = epoch
    print(f"Best validation accuracy updated at epoch: {epoch} as {best_validation_loss}")
ETIME = time.time()
print(f"\n\nSTATS\nTraining done in: {ETIME-STIME:.3f} secs")
print(f"Best Training Loss: {best_training_loss:.3f} at epoch: {best_training_epoch}")
print(f"Best Validation Loss: {best_validation_loss:.3f} at epoch: {best_validation_epoch}")

print(f"Best Training Accuracy: {best_training_accuracy:.3f} at epoch: {best_training_epoch}")
print(f"Best Validation Accuracy: {best_validation_accuracy:.3f} at epoch: {best_validation_epoch}")


Starting Training epoch: 1


Progress - Accu: 0.156	Loss: 2.508: 100%|██████████| 17/17 [00:00<00:00, 32.86it/s]

[TRAIN] Epoch: 1	Accu: 0.15638220134903402	Loss: 2.508178879232968	Time: 0.522 secs
Starting Evaluation epoch: 1


[VALID] Epoch: 1	Accu: 0.15586419900258383	Loss: 2.1139312585194907	Time: 0.378 secs
Learning rate at epoch: 1 is 0.01
Best training loss updated at epoch: 1 as 2.508178879232968
Best validation loss updated at epoch: 1 as 2.1139312585194907
Best training accuracy updated at epoch: 1 as 2.508178879232968
Best validation accuracy updated at epoch: 1 as 2.1139312585194907
Starting Training epoch: 2


Progress - Accu: 0.140	Loss: 2.115: 100%|██████████| 17/17 [00:00<00:00, 32.76it/s]

...

Progress - Accu: 0.484	Loss: 1.304: 100%|██████████| 17/17 [00:00<00:00, 31.61it/s]


[TRAIN] Epoch: 49	Accu: 0.4842113221392912	Loss: 1.3035547733306885	Time: 0.542 secs
Starting Evaluation epoch: 49
[VALID] Epoch: 49	Accu: 0.39120370397965115	Loss: 1.6819366812705994	Time: 0.371 secs
Learning rate at epoch: 49 is 1.9073486328125e-08
Starting Training epoch: 50


Progress - Accu: 0.481	Loss: 1.308: 100%|██████████| 17/17 [00:00<00:00, 31.90it/s]

[TRAIN] Epoch: 50	Accu: 0.4814413785934448	Loss: 1.308019708184635	Time: 0.537 secs
Starting Evaluation epoch: 50


[VALID] Epoch: 50	Accu: 0.39120370397965115	Loss: 1.6819369792938232	Time: 0.366 secs
Learning rate at epoch: 50 is 1.9073486328125e-08


STATS
Training done in: 43.821 secs
Best Training Loss: 1.301 at epoch: 33
Best Validation Loss: 1.636 at epoch: 19
Best Training Accuracy: 0.484 at epoch: 33
Best Validation Accuracy: 0.436 at epoch: 19


In [ ]:
# evaluate on the test dataset